In [1]:
######################################     Modules     #######################################
# MODULES
import sys
from time import process_time
import pickle
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact

sys.path.append('../.')
import verbose_module
sys.path.append('../../../.')
import pIRPgym


computer_name = input("Running experiment on mac? [Y/n]")
if computer_name == '': 
    path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPgym/'
    experiments_path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Column Generation/'
else: 
    path = 'C:/Users/jm.betancourt/Documents/Research/pIRPgym/'
    experiments_path = 'G:/Mi unidad/Research/Supply Chain Analytics/Experiments/Column Generation/'

verbose = True
alphas = [0.1,0.2,0.4,0.6]

Experiments = [f'Experiment {i}' for i in range(1,6)]    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


# Scalability

## By experiments

In [2]:
Experiments_results = {('Instance','M/K'):[5,10,15,20,40],('CG','eff'):[],('CG','gaps'):[]}
Experiments_results.update({(str(alpha),item):[] for alpha in alphas for item in ['eff','gaps']})

for experiment in Experiments:

    reliability = {alpha:[] for alpha in alphas}
    reliability.update({'CG':[]})

    gaps = {alpha:[] for alpha in alphas}
    gaps.update({'CG':[]})

    for replica in range(1,4):
        # Upload results
        with open(experiments_path+f'{experiment}/Replica {replica}/instance_information.pkl', 'rb') as file:
            inst_info = pickle.load(file)

        with open(experiments_path+f'{experiment}/Replica {replica}/CG.pkl', 'rb') as file:
            CG_results = pickle.load(file)
        CG_alpha_results = dict()
        for alpha in alphas: 
            with open(experiments_path+f'{experiment}/Replica {replica}/CG_{alpha}.pkl', 'rb') as file:
                CG_alpha_results[alpha] = pickle.load(file)
        
        # For each day determine the best score
        for t in range(len(CG_results)):
            best_performance = min([CG_results[t][1]]+[CG_alpha_results[alpha][t][1] for alpha in alphas])
            # best_performance = CG_results[t][1]
            # for alpha in alphas: 
            #     if CG_alpha_results[alpha][t][1] < best_performance:
            #         best_performance = CG_alpha_results[alpha][t][1]

            if CG_results[t][1] - best_performance < 0.01:
                reliability['CG'].append(1)
            else:
                reliability['CG'].append(0)
                gaps['CG'].append((CG_results[t][1]-best_performance)/best_performance)

            for alpha in alphas: 
                if CG_alpha_results[alpha][t][1] - best_performance < 0.01:
                    reliability[alpha].append(1)
                else:
                    reliability[alpha].append(0)
                    gaps[alpha].append((CG_alpha_results[alpha][t][1]-best_performance)/best_performance)

    # Storing the results
    Experiments_results[('CG','eff')].append(round(sum(reliability["CG"])/len(reliability["CG"]),4))
    Experiments_results[('CG','gaps')].append(round(sum(gaps["CG"])/len(gaps["CG"]),4))

    for alpha in alphas:
        Experiments_results[(str(alpha),'eff')].append(round(sum(reliability[alpha])/len(reliability[alpha]),4))
        if len(gaps[alpha]) > 0:
            Experiments_results[(str(alpha),'gaps')].append(round(sum(gaps[alpha])/len(gaps[alpha]),4))
        else:
            Experiments_results[(str(alpha),'gaps')].append(0)

df = pd.DataFrame(Experiments_results)
df.index = Experiments
# df.columns = ['M/K','Regular', 'al 0.1','al 0.2','al 0.4','al 0.6']
df = df.style.format(dict.fromkeys(list(df.columns)[1:],"{:.2%}"))
display(df)

In [3]:
sizes = {1:5,2:10,3:15,4:20,5:40}
x_values = [f'Exp{exp} (M/K={size})' for exp,size in sizes.items()]
results = {'CG':{'eff':Experiments_results['CG','eff'], 'gaps':Experiments_results['CG','eff']}}
for alpha in alphas:
    results[alpha] = {'eff':Experiments_results[str(alpha),'eff']}
    results[alpha].update({'gaps':Experiments_results[str(alpha),'gaps']})

def plot_thing(indicator):
    pIRPgym.Visualizations.RoutingV.plot_indicator_evolution(results,indicator,x_axis='Experiments',x_values=x_values)

interact(plot_thing,indicator=widgets.Dropdown(options=['eff','gaps'],value='eff',description='Indicator to visualize:'))

interactive(children=(Dropdown(description='Indicator to visualize:', options=('eff', 'gaps'), value='eff'), O…

<function __main__.plot_thing(indicator)>

## By Suppliers

In [9]:
Experiments_results = {('Instance','M/K'):[],('Instance','Number'):[],('CG','eff'):[],('CG','gaps'):[]}
Experiments_results.update({(str(alpha),item):[] for alpha in alphas for item in ['eff','gaps']})

reliability = {alpha:dict() for alpha in alphas}
reliability.update({'CG':dict()})

gaps = {alpha:dict() for alpha in alphas}
gaps.update({'CG':dict()})

for experiment in Experiments[-1:]:

    for replica in range(1,4):
        # Upload results
        with open(experiments_path+f'{experiment}/Replica {replica}/instance_information.pkl', 'rb') as file:
            inst_info = pickle.load(file)

        with open(experiments_path+f'{experiment}/Replica {replica}/CG.pkl', 'rb') as file:
            CG_results = pickle.load(file)
        CG_alpha_results = dict()
        for alpha in alphas: 
            with open(experiments_path+f'{experiment}/Replica {replica}/CG_{alpha}.pkl', 'rb') as file:
                CG_alpha_results[alpha] = pickle.load(file)

        
        
        # For each day determine the best score
        for t in range(len(CG_results)):
            num_suppliers = len([node for route in inst_info['NN_routes'][t] for node in route if node != 0])
            if num_suppliers not in reliability['CG'].keys():
                reliability['CG'][num_suppliers] = list()
                gaps['CG'][num_suppliers] = list()
                for alpha in alphas:
                    reliability[alpha][num_suppliers] = list()
                    gaps[alpha][num_suppliers] = list()

            best_performance = min([CG_results[t][1]]+[CG_alpha_results[alpha][t][1] for alpha in alphas])

            if CG_results[t][1] - best_performance < 0.01:
                reliability['CG'][num_suppliers].append(1)
            else:
                reliability['CG'][num_suppliers].append(0)
                gaps['CG'][num_suppliers].append((CG_results[t][1]-best_performance)/best_performance)

            for alpha in alphas: 
                if CG_alpha_results[alpha][t][1] - best_performance < 0.01:
                    reliability[alpha][num_suppliers].append(1)
                else:
                    reliability[alpha][num_suppliers].append(0)
                    gaps[alpha][num_suppliers].append((CG_alpha_results[alpha][t][1]-best_performance)/best_performance)

                    

# Storing the results
supplier_categories = sorted(list(set(reliability['CG'].keys())))
Experiments_results[('Instance','M/K')] = supplier_categories

for sup_num in supplier_categories:
    Experiments_results[('Instance','Number')].append(len(reliability["CG"][sup_num]))
    Experiments_results[('CG','eff')].append(round(sum(reliability["CG"][sup_num])/len(reliability["CG"][sup_num]),4))
    if len(gaps['CG'][sup_num]) > 0:
            Experiments_results[('CG','gaps')].append(round(sum(gaps["CG"][sup_num])/len(gaps["CG"][sup_num]),4))
    else:
        Experiments_results[('CG','gaps')].append(0)


    for alpha in alphas:
        Experiments_results[(str(alpha),'eff')].append(round(sum(reliability[alpha][sup_num])/len(reliability[alpha][sup_num]),4))
        if len(gaps[alpha][sup_num]) > 0:
            Experiments_results[(str(alpha),'gaps')].append(round(sum(gaps[alpha][sup_num])/len(gaps[alpha][sup_num]),4))
        else:
            Experiments_results[(str(alpha),'gaps')].append(0)

df = pd.DataFrame(Experiments_results)
df.index = supplier_categories
# df.columns = ['M/K','Regular', 'al 0.1','al 0.2','al 0.4','al 0.6']
df = df.style.format(dict.fromkeys(list(df.columns)[2:],"{:.2%}"))
display(df)

In [10]:
results = {'CG':{'eff':Experiments_results['CG','eff'], 'gaps':Experiments_results['CG','eff']}}
for alpha in alphas:
    results[alpha] = {'eff':Experiments_results[str(alpha),'eff']}
    results[alpha].update({'gaps':Experiments_results[str(alpha),'gaps']})

def plot_thing(indicator):
    pIRPgym.Visualizations.RoutingV.plot_indicator_evolution(results,indicator,x_axis='Experiments',x_values=supplier_categories)

interact(plot_thing,indicator=widgets.Dropdown(options=['eff','gaps'],value='eff',description='Indicator to visualize:'))


interactive(children=(Dropdown(description='Indicator to visualize:', options=('eff', 'gaps'), value='eff'), O…

<function __main__.plot_thing(indicator)>